<br>

# Introdução

Extrai os arquivos baixados em .rar e transforma em .gpkg


In [ ]:
!pip3 install rarfile --upgrade


In [ ]:
import os
import stat
import time
import shutil
import warnings
import urllib.request
import geopandas as gpd
from rarfile import RarFile


In [ ]:
from paths import *


In [ ]:
warnings.simplefilter(action='ignore')

<br>

## Download


In [ ]:
# Read File URLS
input_file = input_path / 'urls.txt'

# Read File
with open(input_file, encoding='utf-8') as f:
    lines = [line.rstrip('\n') for line in f]

# Loop
for line in lines:
    url = urllib.parse.unquote(line)
    #print(url)

    # Filename
    filename = Path(url).name.replace('-', ' ')
    filepath = input_path_brutos / filename
    print(f'> {filename}')

    # Download
    my_headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
    req = urllib.request.Request(line, headers=my_headers)
    with urllib.request.urlopen(req) as r, open(filepath, 'wb') as out_file:
        shutil.copyfileobj(r, out_file)


<br>

## Results


In [ ]:
# List Files
list_files = list(input_path_brutos.rglob('*.rar'))

# Results
print(f'Temos {len(list_files)} arquivos .rar')
list_files


<br>

## Functions

In [ ]:
def unrar_files(rar_filepath, output_path):
    # Unrar data
    with RarFile(rar_filepath) as rarObj:
        rarObj.extractall(output_path)

    # Se extrair pasta, move conteudo da pasta    
    list_files = list(output_path.rglob('*'))
    for file in list_files:
        # a = file.relative_to(output_path)
        if file.is_file():
            try:
                shutil.move(file, output_path)
            except Exception as e:
                print(e)


In [ ]:
temp_path = input_path / 'temp'


In [ ]:
def write_geodata(gdf, output_path, filename):
    gdf.to_file(
        output_path / 'comite_alto_tiete.gpkg',
        layer=f'{filename}',
        driver='GPKG'
    )

    gdf.to_file(
        output_path / f'{filename}.gpkg',
        layer=f'{filename}',
        driver='GPKG'
    )


In [ ]:
def delete_path(path, msg):
    try:
        os.chmod(path, 0o777)
        shutil.rmtree(
            path,
            onerror=print(msg),
            ignore_errors=True,
        )
    except Exception as e:
        print(e)

In [ ]:
def clean_thrash(path):
    if path.is_dir():
        list_files = list(path.rglob('*'))
        for f in list_files:
            if f.is_file():
                os.chmod(f , stat.S_IWRITE)
                os.remove(f)
            if f.is_dir():
                clean_thrash(f)
                os.chmod(f , stat.S_IWRITE)            
                f.unlink()
    else:
        print('Não é dir')


In [ ]:
delete_path(temp_path, 'dds')

In [ ]:
clean_thrash(temp_path)

In [ ]:
for file in list_files:
    print('-'*40)
    print(f'{file}')
    rar_filepath = file

    # Delete Path
    delete_path(temp_path, 'Erro pra Deletar!')
    clean_thrash(temp_path)

    # Set path to work
    temp_path = input_path / 'temp'
    temp_path.mkdir(exist_ok=True)

    # Unrar data
    unrar_files(rar_filepath, temp_path)

    # Get list of shapefiles
    list_shps = list(temp_path.rglob('*.shp'))

    if len(list_shps) == 1:
        # Shapefile
        shp_file = list_shps[0]
        print(shp_file)

        # Set name for output
        shp_file_out = shp_file.stem.lower()
        print(shp_file_out)

        # Read Shapefile
        if shp_file_out == '45_subbacias':
            gdf = gpd.read_file(shp_file, encoding='ISO-8859-1')
            gdf = gdf.to_crs(epsg=4326)
            dict_fix = {
                'CrÃ\xadtica': 'Crítica',
                'NÃ£o CrÃ\xadtica': 'Não Crítica',
                'Sem ClassificaÃ§Ã£o': 'Sem Classificação'
            }
            gdf.replace(
                {
                    'Crit_IQA': dict_fix,
                    'Crit_Uso': dict_fix,
                    'IAP': dict_fix,
                    'IVA': dict_fix,
                    'OD': dict_fix,
                },
                inplace=True
            )

        else:
            gdf = gpd.read_file(temp_path / shp_file)
            gdf = gdf.to_crs(epsg=4326)

        # Write data
        write_geodata(gdf, output_path_gpkg, shp_file_out)

        # Delete Path
        delete_path(temp_path, 'Erro pra Deletar!')
        clean_thrash(temp_path)


    else:
        print('Erro: Lista de shapes tem mais de um!')
        print(list_shps)

# Results
print('Fimmmm')

In [ ]:
delete_path(temp_path, 'Erro pra Deletar!')


<br>

## Convert 7zip

In [ ]:
from open_geodata import converts

converts.files.convert_to_7zip(output_path_gpkg, output_path_geo)